In [1]:
import os
import pandas as pd

# 파일 경로 설정
folder_path = r"C:\VISSIM_Workspace\network_new\test"
inpx_name = "network01_mer"  # 수정 필요
destination_dir = os.path.join(folder_path, inpx_name)
os.makedirs(destination_dir, exist_ok=True)

# .mer 파일 필터링
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]
print("총 .mer 파일 수:", len(mer_list))

# 시뮬레이션 단위 설정
siml_count = 10
mer_count = 0

for start in range(0, len(mer_list), siml_count):
    batch_files = mer_list[start:start + siml_count]
    df_list = []

    for file in batch_files:
        mer_path = os.path.join(destination_dir, file)
        print("처리 중:", mer_path)

        with open(mer_path, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        data_start_idx = None
        for j, line in enumerate(lines):
            if "Measurem." in line:
                data_start_idx = j
                break

        if data_start_idx is not None:
            columns = lines[data_start_idx].strip().split(";")
            data_lines = lines[data_start_idx + 1:]
            data = [line.strip().split(";") for line in data_lines if line.strip()]
            df = pd.DataFrame(data, columns=columns)
            df.columns = df.columns.str.strip()

            # 숫자형 컬럼 변환
            numeric_columns = ["Measurem.", "t(Entry)", "t(Exit)", "VehNo", "Vehicle type",
                               "Line", "v[km/h]", "b[m/s2]", "Occ", "Pers", "tQueue", "VehLength[m]"]

            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce")

            # 시간 컬럼 계산 (원본 유지)
            df["time"] = df[["t(Entry)", "t(Exit)"]].max(axis=1)

            # 평균 계산
            df = df.groupby(["Vehicle type", "Measurem.", "time"])[["b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]] \
                   .mean().reset_index().round(2)

            df_list.append(df)

    # 배치 병합 및 평균
    merged_df = pd.concat(df_list, ignore_index=True)
    avg_df = merged_df.groupby(["Vehicle type", "Measurem.", "time"])[["b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]] \
                      .mean().reset_index().round(2)

    # 출력 경로 생성
    merout_path = os.path.join(destination_dir, "vehicle_output")
    os.makedirs(merout_path, exist_ok=True)

    # 파일명 생성
    xlsx_files = sorted([f for f in os.listdir(merout_path) if f.endswith(".xlsx")],
                        key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    last_file = int(xlsx_files[-1].split("_")[0]) + 1 if xlsx_files else 1
    excel_path = os.path.join(merout_path, f"{last_file}_vehicle_output.xlsx")

    # 저장
    avg_df.to_excel(excel_path, index=False)
    print("생성 완료:", excel_path)

    mer_count += 1
    if mer_count >= len(mer_list) / siml_count:
        print("시뮬레이션 분석이 완료되었습니다.")
        break


총 .mer 파일 수: 1
처리 중: C:\VISSIM_Workspace\network_new\test\network01_mer\network02_수정_001.mer
생성 완료: C:\VISSIM_Workspace\network_new\test\network01_mer\vehicle_output\2_vehicle_output.xlsx
시뮬레이션 분석이 완료되었습니다.
